In [6]:
from abc import ABC, abstractmethod
import numpy as np
from scipy.stats import multivariate_normal

class Model(ABC):
    @abstractmethod
    def train(self, train_inputs, train_labels):
        pass
    
    @abstractmethod
    def predict(self, test_inputs):
        pass
        
class DiagonalGaussian(Model):
    def train(self, train_inputs, train_labels):
        self.mean = np.mean(train_inputs, axis=0)
        self.covariance = np.diag(np.sum((train_inputs - self.mean) ** 2, axis=0) / train_inputs.shape[0])
        #self.covariance = np.diag(np.diag(np.cov(train_inputs.T)))

        #  Test
        print(self.covariance - np.diag(np.diag(np.cov(train_inputs.T))) <= 0.00000000000001, "\n")
        
        self.inv_covariance = np.linalg.inv(self.covariance)
        self.first_half_pdf = 1 / (np.sqrt((2 * np.pi) ** train_inputs.shape[1] * np.linalg.det(self.covariance)))
    
    def __logpdf__(self, x):
        diff = x - self.mean
        second_half_pdf = np.exp(-0.5 * np.dot(np.dot(diff, self.inv_covariance), diff))
        return np.log(self.first_half_pdf * second_half_pdf)
    
    def predict(self, test_inputs):
        predictions = np.zeros((test_inputs.shape[0], 2))
        for i, x in enumerate(test_inputs):
            predictions[i, 0] = multivariate_normal.logpdf(x, mean=self.mean, cov=self.covariance)
            predictions[i, 1] = self.__logpdf__(x)
            
        return predictions

class ParzenIsotropicGaussian(Model):
    def __init__(self, sigma = 5):
        self.sigma = sigma
        
    def train(self, train_inputs, train_labels):
        self.train_inputs = train_inputs
        self.train_labels = train_labels
        
        self.covariance = np.identity(train_inputs.shape[1]) * self.sigma
        self.inv_covariance = np.linalg.inv(self.covariance)
        d = train_inputs.shape[1]
        self.first_half_pdf = 1 / (np.sqrt((2 * np.pi * self.sigma) ** d))
        #self.first_half_pdf = 1 / (np.sqrt((2 * np.pi) ** train_inputs.shape[1] * np.linalg.det(self.covariance)))
    
    def __logpdf__(self, x, mean):
        diff = x - mean
        second_half_pdf = np.exp(-0.5 * np.linalg.norm(diff) ** 2 / self.sigma)
        #second_half_pdf = np.exp(-0.5 * np.dot(np.dot(diff, self.inv_covariance), diff))
        return np.log(self.first_half_pdf * second_half_pdf)

    def predict(self, test_inputs):
        predictions = np.zeros((test_inputs.shape[0], 2))
        for i, x in enumerate(test_inputs):
            for X in train_inputs:
                predictions[i, 0] += multivariate_normal.logpdf(x, mean=X, cov=self.sigma)
                predictions[i, 1] += self.__logpdf__(x, X)
                
        predictions /= train_inputs.shape[0]
        return predictions

# Test
data = np.loadtxt('iris.txt')
train_inputs = data[:40, :4]
train_labels = data[:40, 4]

dg = DiagonalGaussian()
dg.train(train_inputs, train_labels)

parzen = ParzenIsotropicGaussian()
parzen.train(train_inputs, train_labels)

dg_prediction = dg.predict(data[40:50, :4])
parzen_prediction = parzen.predict(data[40:50, :4])

print(dg_prediction[:, 0] - dg_prediction[:, 1] <= 0.00000000000001, "\n")
print(parzen_prediction[:, 0] - parzen_prediction[:, 1] <= 0.00000000000001, "\n")

print(dg_prediction, "\n")
print(parzen_prediction)

[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]] 

[ True  True  True  True  True  True  True  True  True  True] 

[ True  True  True  True  True  True  True  True  True  True] 

[[ 1.81164841  1.81164841]
 [-4.53350041 -4.53350041]
 [ 0.14727078  0.14727078]
 [-5.03872326 -5.03872326]
 [-2.81115412 -2.81115412]
 [ 1.17988159  1.17988159]
 [ 1.60084375  1.60084375]
 [ 1.36876382  1.36876382]
 [ 1.89230936  1.89230936]
 [ 2.27131508  2.27131508]] 

[[-6.92727996 -6.92727996]
 [-7.08862996 -7.08862996]
 [-6.97362996 -6.97362996]
 [-6.93957996 -6.93957996]
 [-6.95847996 -6.95847996]
 [-6.95082996 -6.95082996]
 [-6.93847996 -6.93847996]
 [-6.94992996 -6.94992996]
 [-6.93722996 -6.93722996]
 [-6.92687996 -6.92687996]]
